<h1>Simulating continous portfolio optimization</h1>

In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
#stocks = ['AAPL', 'AMZN', 'MSFT', 'TSLA']
stocks = ['AAPL', 'TSLA']

In [3]:
# Download data
data_df = yf.download(stocks, start='2018-01-01')

# Convert the index to datetime
data_df.index = pd.to_datetime(data_df.index)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


In [4]:
display(data_df)

Price            Close                    High                     Low  \
Ticker            AAPL        TSLA        AAPL        TSLA        AAPL   
Date                                                                     
2018-01-02   40.479828   21.368668   40.489230   21.474001   39.774850   
2018-01-03   40.472794   21.150000   41.017978   21.683332   40.409348   
2018-01-04   40.660778   20.974667   40.764175   21.236668   40.437536   
2018-01-05   41.123722   21.105333   41.210668   21.149332   40.665487   
2018-01-08   40.970974   22.427334   41.267063   22.468000   40.872274   
...                ...         ...         ...         ...         ...   
2025-04-02  223.889999  282.760010  225.190002  284.989990  221.020004   
2025-04-03  203.190002  267.279999  207.490005  276.299988  201.250000   
2025-04-04  188.380005  239.429993  199.880005  261.000000  187.339996   
2025-04-07  181.460007  233.289993  194.149994  252.000000  174.619995   
2025-04-08  172.419998  221.860001  190.339996  250.440002  169.210007   

Price                         Open                 Volume             
Ticker            TSLA        AAPL        TSLA       AAPL       TSLA  
Date                                                                  
2018-01-02   20.733334   39.986346   20.799999  102223600   65283000  
2018-01-03   21.036667   40.543292   21.400000  118071600   67822500  
2018-01-04   20.378668   40.545630   20.858000   89738400  149194500  
2018-01-05   20.799999   40.757134   21.108000   94640000   68868000  
2018-01-08   21.033333   40.970974   21.066668   82271200  147891000  
...                ...         ...         ...        ...        ...  
2025-04-02  251.270004  221.320007  254.600006   35905900  212787800  
2025-04-03  261.510010  205.539993  265.290009  103419000  136174300  
2025-04-04  236.000000  193.889999  255.380005  125910900  181229400  
2025-04-07  214.250000  177.199997  223.779999  160466300  183453800  
2025-04-08  217.800003  186.699997  245.000000  120363400  170068600  

[1827 rows x 10 columns]

In [5]:
def get_data_before(data_df: pd.DataFrame, end: str):
    filtered_df = data_df[data_df.index <= end]
    return filtered_df

In [6]:
def optimize_portfolio(data_df, stocks: list, end: str):
    current_data_df = get_data_before(data_df, end)
    
    # Calculating daily returns
    current_data_df = current_data_df['Close']
    x = current_data_df.pct_change()

    # Storing the weights, returns and Sharpe ratios for each portfolio
    p_weights, p_returns, p_risk, p_sharpe = [], [], [], []

    # Running a for loop, generate the random weights and calculate the returns, volatility and Sharpe ratio of the portfolio.
    count = 5000
    for k in range(0, count):
        # Randomly assign a weight to each stock in our portfolio, and then calculate the metrics for that portfolio, including the Sharpe ratio.
        wts = np.random.uniform(size = len(stocks))
        wts = wts/np.sum(wts)
        p_weights.append(wts)
    
        # Returns
        mean_ret = (x.mean() * wts).sum()*252
        p_returns.append(mean_ret)
        
        # Volatility
        ret = (x * wts).sum(axis = 1)
        annual_std = np.std(ret) * np.sqrt(252)
        p_risk.append(annual_std)
            
        # Sharpe ratio
        sharpe = (np.mean(ret) / np.std(ret))*np.sqrt(252)
        p_sharpe.append(sharpe)

    # Finding the optimal index
    max_ind = np.argmax(p_sharpe)

    # Finding the max sharpe ratio
    max_sharpe_ratio = p_sharpe[max_ind]
    
    # Finding the optimal stock weights
    optimal_stock_weights = np.round(p_weights[max_ind], 2)

    return round(float(max_sharpe_ratio), 2), optimal_stock_weights.tolist()

In [7]:
max_sharpe_ratio, optimal_stock_weights = optimize_portfolio(
    data_df=data_df,
    stocks=stocks, 
    end='2025-04-01'
)

# Max Sharpe ratio
print("The maximum sharpe ratio: ", max_sharpe_ratio)
# Stocks
print("The stocks: ", stocks)
# Stock Weights
print("The optimal stock weights that gives the maximum sharpe ratio: ", optimal_stock_weights)

The maximum sharpe ratio:  1.06
The stocks:  ['AAPL', 'TSLA']
The optimal stock weights that gives the maximum sharpe ratio:  [0.72, 0.28]


In [8]:
first_max_sharpe_ratio = max_sharpe_ratio
first_optimal_stock_weights = optimal_stock_weights
allowed_difference = 0.05

for i in range(10):
    print(f"Test Run nr.", i+1)
    
    difference_found = False
    
    test_max_sharpe_ratio, test_optimal_stock_weights = optimize_portfolio(
        data_df=data_df,
        stocks=stocks, 
        end='2025-04-01'
    )
    
    for test_index in range(len(stocks)):
        test_value = first_optimal_stock_weights[test_index]
        if abs(test_optimal_stock_weights[test_index] - test_value) > allowed_difference:
            difference_found = True
            print(f"Share difference for {stocks[test_index]} is bigger than {allowed_difference} for the same optimization run")
            print(f"First Optimal Stock Weight = {test_value}, and Current Optimal Stock Weight = {test_optimal_stock_weights[test_index]}")

    if not difference_found:
        print("No big difference was found.")
        
    print("----------------------------------")

print("Testing is done.")

Test Run nr. 1
No big difference was found.
----------------------------------
Test Run nr. 2
No big difference was found.
----------------------------------
Test Run nr. 3
No big difference was found.
----------------------------------
Test Run nr. 4
No big difference was found.
----------------------------------
Test Run nr. 5
No big difference was found.
----------------------------------
Test Run nr. 6
No big difference was found.
----------------------------------
Test Run nr. 7
No big difference was found.
----------------------------------
Test Run nr. 8
No big difference was found.
----------------------------------
Test Run nr. 9
No big difference was found.
----------------------------------
Test Run nr. 10
No big difference was found.
----------------------------------
Testing is done.


In [9]:
from datetime import datetime, timedelta

simulation_optimal_weights = {}

# Start and end dates
start_date = datetime(2025, 4, 1)
end_date = datetime(2025, 4, 7)

# Loop through each date
current_date = start_date
while current_date <= end_date:
    current_date_str = current_date.strftime('%Y-%m-%d')
    
    max_sharpe_ratio, optimal_stock_weights = optimize_portfolio(
        data_df=data_df,
        stocks=stocks, 
        end=current_date_str
    )

    simulation_optimal_weights[current_date_str] = {
        "max_sharpe_ratio": max_sharpe_ratio,
        "optimal_stock_weights": {}
    }

    for stock_index in range(len(stocks)):
        simulation_optimal_weights[current_date_str]["optimal_stock_weights"][stocks[stock_index]] = optimal_stock_weights[stock_index]

    print(f"{current_date_str = }")
    print(f"{stocks = }")
    print(f"{max_sharpe_ratio = }")
    print(f"{optimal_stock_weights = }")
    print("Sum of weights: ", round(sum(optimal_stock_weights)))
    print("------------------------")

    current_date += timedelta(days=1)

current_date_str = '2025-04-01'
stocks = ['AAPL', 'TSLA']
max_sharpe_ratio = 1.06
optimal_stock_weights = [0.72, 0.28]
Sum of weights:  1
------------------------
current_date_str = '2025-04-02'
stocks = ['AAPL', 'TSLA']
max_sharpe_ratio = 1.06
optimal_stock_weights = [0.71, 0.29]
Sum of weights:  1
------------------------
current_date_str = '2025-04-03'
stocks = ['AAPL', 'TSLA']
max_sharpe_ratio = 1.03
optimal_stock_weights = [0.69, 0.31]
Sum of weights:  1
------------------------
current_date_str = '2025-04-04'
stocks = ['AAPL', 'TSLA']
max_sharpe_ratio = 0.99
optimal_stock_weights = [0.68, 0.32]
Sum of weights:  1
------------------------
current_date_str = '2025-04-05'
stocks = ['AAPL', 'TSLA']
max_sharpe_ratio = 0.99
optimal_stock_weights = [0.68, 0.32]
Sum of weights:  1
------------------------
current_date_str = '2025-04-06'
stocks = ['AAPL', 'TSLA']
max_sharpe_ratio = 0.99
optimal_stock_weights = [0.68, 0.32]
Sum of weights:  1
------------------------
current_date_str = '20

In [10]:
def get_prices_at_date(
    data_df: pd.DataFrame, 
    date: str, 
    price_type: str
) -> dict[tuple, float]:
    
    filtered_df = data_df[data_df.index == date]

    # Initialize an empty dictionary
    result_dict = {}
    # Loop over the columns
    for col in filtered_df.columns:
        if price_type in col:
            price, ticker = col  # Extract Price and Ticker from the MultiIndex
            if ticker not in result_dict:
                result_dict[ticker] = {}  # Initialize nested dictionary for each Ticker
            result_dict[ticker][price] = filtered_df[col].tolist()  # Assign the corresponding values to the dictionary
            
    return result_dict

In [11]:
def buy(
    data_df: pd.DataFrame, 
    date: str, 
    optimal_stock_weights: dict[str, float],
    balance: float
) -> (dict[str, float] | None, float | None, float | None):
    
    price_type = "Open"
    
    current_prices = get_prices_at_date(
        data_df=data_df, 
        date=date, 
        price_type=price_type
    )

    current_shareholding_value = 0
    input_balance = balance
    current_shareholding = {}

    min_balance_required = 0
    for stock, price in current_prices.items():
        min_balance_required += price[price_type][0]
    if input_balance < min_balance_required:
        return None, None, balance
    
    for stock, price in current_prices.items():
        optimized_stock_weight = optimal_stock_weights[stock]
        buy_total_amount = input_balance * optimized_stock_weight
        if buy_total_amount > balance or balance == 0:
            return None, None, balance
        else:
            buy_stock_price = price[price_type][0]
            current_shareholding[stock] = round( buy_total_amount / buy_stock_price, 2)
            current_shareholding_value += buy_total_amount
            balance -= buy_total_amount

    return current_shareholding, round(current_shareholding_value, 2), round(balance, 2)

In [12]:
balance = 1000.00 # in Dollars
shareholding = {} # in Shares
shareholding_value = 0 # in Dollars

In [24]:
current_shareholding, current_shareholding_value, current_balance = buy(
    data_df=data_df, 
    date="2025-04-01", 
    optimal_stock_weights=simulation_optimal_weights['2025-04-01']['optimal_stock_weights'], 
    balance=balance
)

shareholding = current_shareholding if current_shareholding else shareholding
shareholding_value = current_shareholding_value if current_shareholding_value else shareholding_value
balance = current_balance

print(f"{balance = }") # Dollars
print(f"{shareholding = }") # Shares
print(f"{shareholding_value = }") # Dollars

balance = 0.0
shareholding = {'AAPL': 3.28, 'TSLA': 1.06}
shareholding_value = 1000.0


In [31]:
def sell(
    data_df: pd.DataFrame, 
    date: str, 
    shareholding: dict[str, float], 
    balance: int
) -> (dict[str, float], int, int):

    price_type = "Close"
    
    current_prices = get_prices_at_date(
        data_df=data_df, 
        date=date, 
        price_type=price_type
    )

    shareholding_value = 0
    for stock, share in shareholding.items():
        price = current_prices[stock]
        sell_amount = round(share*price[price_type][0], 2)
        shareholding_value -= sell_amount
        balance += sell_amount
        shareholding[stock] = 0
    
    return current_shareholding, round(current_shareholding_value, 2), round(balance, 2)

In [33]:
current_shareholding, current_shareholding_value, current_balance = sell(
    data_df=data_df, 
    date="2025-04-01", 
    shareholding=shareholding, 
    balance=balance
)

shareholding = current_shareholding if current_shareholding else shareholding
shareholding_value = current_shareholding_value if current_shareholding_value else shareholding_value
balance = current_balance

print(f"{balance = }") # Dollars
print(f"{shareholding = }") # Shares
print(f"{shareholding_value = }") # Dollars

TypeError: type NoneType doesn't define __round__ method

In [94]:
def sell(data_df: pd.DataFrame, date: str, shareholding: dict[str, float], current_balance: int) -> (dict[str, float], int, int):
    current_prices = get_prices_at_date(data_df=data_df, date=date)

    current_shareholding = {}
    shareholding_value = 0
    for stock, share in shareholding.items():
        price = current_prices[stock]
        sell_amount = round(share*price['Close'][0], 2)
        shareholding_value -= sell_amount
        current_balance += sell_amount
        current_shareholding[stock] = 0
    
    return current_shareholding, round(shareholding_value), round(current_balance)

In [100]:
current_shareholding, shareholding_value, current_balance = sell(
    data_df=data_df, 
    date="2025-04-01", 
    shareholding=current_shareholding,
    current_balance=balance
)

balance = current_balance

print(f"{current_balance = }") # Dollars
print(f"{current_shareholding = }") # Shares
print(f"{shareholding_value = }") # Dollars

current_balance = 1000
current_shareholding = {'AAPL': 0, 'TSLA': 0}
shareholding_value = 0
